In [5]:
import pandas as pd
import numpy as np

In [2]:
filepath = ['climdiv-cddcdv-v1.0.0-20250806',
            'climdiv-hddcdv-v1.0.0-20250806',
            'climdiv-pcpndv-v1.0.0-20250806',
            'climdiv-pdsidv-v1.0.0-20250806',
            'climdiv-phdidv-v1.0.0-20250806',
            'climdiv-pmdidv-v1.0.0-20250806',
            'climdiv-sp01dv-v1.0.0-20250806',
            'climdiv-sp02dv-v1.0.0-20250806',
            'climdiv-sp03dv-v1.0.0-20250806',
            'climdiv-sp06dv-v1.0.0-20250806',
            'climdiv-sp09dv-v1.0.0-20250806',
            'climdiv-sp12dv-v1.0.0-20250806',
            'climdiv-sp24dv-v1.0.0-20250806',
            'climdiv-tmaxdv-v1.0.0-20250806',
            'climdiv-tmindv-v1.0.0-20250806',
            'climdiv-tmpcdv-v1.0.0-20250806',
            'climdiv-zndxdv-v1.0.0-20250806']

value_names = ['cddc','hddc','pcpn','pdsi','phdi','pmdi','sp01','sp02','sp03','sp06','sp09','sp12','sp24','tmax','tmin','tmpc','zndx']

colspecs = [(0,2), (2,4),(4,6),(6,10)] + [(10 + i*7, 17 + i*7) for i in range(12)]
colnames = ['State-Code', 'Division','Element-Code','Year'] + [f'Month_{i+1}' for i in range(12)]

data_dict = {}

for path, value_name in zip(filepath, value_names):
    df = pd.read_fwf(path, colspecs=colspecs, names=colnames, 
                 converters = {'State-Code':lambda x: str(x).zfill(2),
                               'Division':lambda x: str(x).zfill(2),
                               'Element-Code':lambda x: str(x).zfill(2)})
    df_co = df[df['State-Code'] == "05"]

    df_co_long = df_co.melt(id_vars=['State-Code','Division','Element-Code','Year'],
                            value_vars=[f'Month_{i+1}' for i in range(12)],
                            var_name='Month',
                            value_name= value_name)
    
    data_dict[path]=df_co_long

In [ ]:
data_merged = data_dict['climdiv-cddcdv-v1.0.0-20250806']

for path, vname in zip(filepath[1:],value_names[1:]):
    data_merged = pd.merge(data_merged,data_dict[path][['State-Code','Division','Year','Month',vname]],
                           on=['State-Code','Division','Year','Month'],how='left' )

data_merged = data_merged.drop('Element-Code',axis=1)

data_merged['Month'] = data_merged['Month'].str.extract(r'Month_(\d+)').astype(int)
data_merged['Day'] = 1
data_merged['Date'] = pd.to_datetime(data_merged[['Year','Month','Day']])
data_merged = data_merged.drop(columns=['Year','Month','Day'])
data_merged = data_merged.sort_values(by=['Division','Date']).reset_index(drop=True)

column_move = data_merged.pop('Date')
data_merged.insert(2,'Date',column_move)

data_merged.replace([-99.99, -99.90, -9.99, -9999], np.nan, inplace=True)

In [8]:
data_merged.to_csv('NOAA_Data_Final.csv',index=False, encoding='utf-8')